### Update runtime

In [ ]:
import os
import requests

qadfUrl = 'https://raw.githubusercontent.com/vlad-yeghiazaryan/QADF/master/quantileADF.py'
reportUrl = 'https://raw.githubusercontent.com/vlad-yeghiazaryan/QADF/master/report.py'
files = [qadfUrl, reportUrl]

# Loading python files into environment
for file in files:
    with open(file.split('/')[-1], 'w') as f:
        f.write(requests.get(file).text)

# Install updates
!pip install statsmodels - -upgrade

# Restart
def restart_runtime():
    os.kill(os.getpid(), 9)

restart_runtime()

### Setup

In [1]:
# Main
import numpy as np
import pandas as pd

# Unit root
from quantileADF import QADF
from report import countryReport, reportCountries

# Utilities
import warnings
from statsmodels.tools.sm_exceptions import IterationLimitWarning
warnings.simplefilter('ignore', IterationLimitWarning)
pd.set_option('display.max_rows', 500)

# Setting up Urls
repoUrl = 'https://raw.githubusercontent.com/vlad-yeghiazaryan/QADF/master/data/'
datasetUrl = repoUrl + 'dataset.csv'
broadREERsUrl = repoUrl + 'broadREERs.csv'
narrowREERsUrl = repoUrl + 'narrowREERs.csv'

### Report

In [2]:
broadREERs = pd.read_csv(broadREERsUrl, index_col=0, parse_dates=True)
narrowREERs = pd.read_csv(narrowREERsUrl, index_col=0, parse_dates=True)

# Extracting country information
countries = ['Armenia', 'Belarus', 'Kazakhstan', 'Kyrgyz Republic',
             'Moldova', 'Russian Federation', 'Tajikistan', 'Ukraine']

startDate = '2000-01-01'
startDateWithoutTaj = '1995-01-01'

dataNarrow =  narrowREERs[countries].loc[startDate:]
dataNarrow.index.freq = 'MS'

data = broadREERs[countries].loc[startDate:]
data.index.freq = 'MS'

data.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 251 entries, 2000-01-01 to 2020-11-01
Freq: MS
Data columns (total 8 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   Armenia             251 non-null    float64
 1   Belarus             251 non-null    float64
 2   Kazakhstan          251 non-null    float64
 3   Kyrgyz Republic     251 non-null    float64
 4   Moldova             251 non-null    float64
 5   Russian Federation  251 non-null    float64
 6   Tajikistan          251 non-null    float64
 7   Ukraine             251 non-null    float64
dtypes: float64(8)
memory usage: 17.6 KB


In [3]:
data.head(2)

,Armenia,Belarus,Kazakhstan,Kyrgyz Republic,Moldova,Russian Federation,Tajikistan,Ukraine
Date,,,,,,,,
2000-01-01,67.584163,121.249657,77.580923,76.395403,67.088115,47.243677,110.087088,74.340017
2000-02-01,68.494432,109.552717,78.210787,76.987894,66.917751,47.791244,109.455091,75.456058


In [4]:
levels=[0.01,0.05,0.1]
quantiles = np.arange(0.1, 1, 0.1)
drop = ['δ²', 'Lags', 'ρ₁(OLS)', 'CV1%', 'CV5%', 'CV10%']
qadf = QADF(model='c', pmax=12, ic='AIC')
reps = 1

In [5]:
# Broad EAEU levels
results, report = reportCountries(data, qadf, quantiles, repetitions=reps, 
                                  significanceLevels=levels, dropColumns=drop)

# # Broad EAEU logs
# logResults, logReport = reportCountries(data.apply(np.log, axis=1), qadf, quantiles, repetitions=reps, 
#                                   significanceLevels=levels, dropColumns=drop)

# # Narrow EAEU levels
# narrowResults, narrowReport = reportCountries(dataNarrow, qadf, quantiles, repetitions=reps, 
#                                   significanceLevels=levels, dropColumns=drop)

# # Narrow EAEU logs
# logNarrowResults, logNarrowReport = reportCountries(dataNarrow.apply(np.log, axis=1), qadf, quantiles, 
#                                                     repetitions=reps, significanceLevels=levels, dropColumns=drop)

Starting Execution:

Armenia finished in: 3.68s
Belarus finished in: 3.36s
Kazakhstan finished in: 2.86s
Kyrgyz Republic finished in: 2.12s
Moldova finished in: 1.58s
Russian Federation finished in: 2.21s
Tajikistan finished in: 1.5s
Ukraine finished in: 2.18s

Total time spent executing: 0m, 21s


In [8]:
report

quantile                                         0.1            0.2  \
Countries          Variable/Quantile                                  
Armenia            α₀(τ)                 0.7(0.0)***   0.09(0.0)***   
                   ρ₁(τ)                0.97(0.0)***   0.99(0.0)***   
                   Half-lives (years)              2              4   
                   tₙ(τ)                -1.8(0.0)***  -1.15(0.0)***   
                   QKS                      1.8(1.0)                  
Belarus            α₀(τ)               -1.79(0.0)***   0.07(0.0)***   
                   ρ₁(τ)                    1.0(1.0)   0.98(0.0)***   
                   Half-lives (years)             25              4   
                   tₙ(τ)                   -0.1(1.0)  -1.39(0.0)***   
                   QKS                     1.83(1.0)                  
Kazakhstan         α₀(τ)                0.93(0.0)***   0.98(0.0)***   
                   ρ₁(τ)                0.97(0.0)***   0.98(0.0)***   
                   Half-lives (years)              2              3   
                   tₙ(τ)               -1.04(0.0)***  -1.62(0.0)***   
                   QKS                  2.65(0.0)***                  
Kyrgyz Republic    α₀(τ)                4.25(0.0)***   2.08(0.0)***   
                   ρ₁(τ)                0.94(0.0)***   0.97(0.0)***   
                   Half-lives (years)              1              2   
                   tₙ(τ)               -2.54(0.0)***   -2.2(0.0)***   
                   QKS                  2.54(0.0)***                  
Moldova            α₀(τ)                0.62(0.0)***    0.7(0.0)***   
                   ρ₁(τ)                   0.97(1.0)      0.98(1.0)   
                   Half-lives (years)              2              3   
                   tₙ(τ)                  -1.73(1.0)  -1.88(0.0)***   
                   QKS                     1.88(1.0)                  
Russian Federation α₀(τ)                1.73(0.0)***    1.3(0.0)***   
                   ρ₁(τ)                   0.96(1.0)      0.97(1.0)   
                   Half-lives (years)              1              2   
                   tₙ(τ)                  -1.42(1.0)     -1.84(1.0)   
                   QKS                     2.08(1.0)                  
Tajikistan         α₀(τ)                5.27(0.0)***   3.65(0.0)***   
                   ρ₁(τ)                0.92(0.0)***   0.95(0.0)***   
                   Half-lives (years)              1              1   
                   tₙ(τ)               -3.93(0.0)***  -3.05(0.0)***   
                   QKS                  3.93(0.0)***                  
Ukraine            α₀(τ)                -2.3(0.0)***   -0.1(0.0)***   
                   ρ₁(τ)                    1.0(1.0)      0.99(1.0)   
                   Half-lives (years)              ∞              6   
                   tₙ(τ)                   0.06(1.0)     -0.56(1.0)   
                   QKS                  2.68(0.0)***                  

quantile                                         0.3            0.4  \
Countries          Variable/Quantile                                  
Armenia            α₀(τ)                0.51(0.0)***  -0.19(0.0)***   
                   ρ₁(τ)                   0.99(1.0)       1.0(1.0)   
                   Half-lives (years)              4             66   
                   tₙ(τ)                  -1.52(1.0)     -0.09(1.0)   
                   QKS                                                
Belarus            α₀(τ)                0.57(0.0)***   1.04(0.0)***   
                   ρ₁(τ)                0.99(0.0)***   0.98(0.0)***   
                   Half-lives (years)              4              4   
                   tₙ(τ)                  -1.42(1.0)  -1.83(0.0)***   
                   QKS                                                
Kazakhstan         α₀(τ)                1.31(0.0)***   0.54(0.0)***   
                   ρ₁(τ)                0.98(0.0)***   0.99(0.0)***   
                   Half

In [9]:
report.to_excel('Outputs/report.xlsx', float_format="%.3f")